In [8]:
import os
import io
import json
import pandas as pd

In [9]:
def str2list(list_str):
    temp_list = list_str.split(',')
    for i, word in enumerate(temp_list):
        if i == 0:
            word = word[1:]
        elif i == len(temp_list)-1:
            word = word[:-1]
        word2int = int(word)
        temp_list[i] = word2int
    return temp_list

def str22dlist(list_str):
    temp_list = list_str.split(']')
    corr_2dlist = []
    for i, word in enumerate(temp_list):
        if len(word) > 0:
            corr_str = word.split(',')
            corr_list = []
            for j, corr in enumerate(corr_str):
                if j == 0:
                    if i == 0:
                        corr = corr[2:]
                    else:
                        corr = corr[1:]
                corr_list.append(int(corr))
            corr_2dlist.append(corr_list)
    return corr_2dlist

In [10]:
key_name = ['measure_value', 'measure_name', 'time', 'Frame', 'OCR_ID', 'Class', 'ID', 'File']
video_name = 'query_response'
kp_label = {'legbkp': 'Leg_back', 'legfkp': 'Leg_front', 'spinekp': 'Spine', 'headkp': 'Head', 'tailkp': 'Tail'}

with open('json_data_v3/'+video_name+'.json') as TSjson:
    TSjson = json.load(TSjson)
    bbox_data_list = []
    kp_data_list = []
    for i, row in enumerate(TSjson['Rows']):
        if i < len(TSjson['Rows']):
            row_data = row["Data"]
            if row_data[1]["ScalarValue"] == 'BB':
                bbox_coor = row_data[0]["ScalarValue"]
                bbox_coor = str2list(bbox_coor)
                bbox_data = [int(row_data[3]["ScalarValue"])-1, row_data[5]["ScalarValue"], 
                             int(row_data[6]["ScalarValue"]), bbox_coor[0], 
                             bbox_coor[1], bbox_coor[2], 
                             bbox_coor[3], row_data[4]["ScalarValue"],
                             row_data[2]["ScalarValue"], 1.0]
                bbox_data_list.append(bbox_data)
            elif row_data[1]["ScalarValue"] == 'KP':
                kp_coor = row_data[0]["ScalarValue"]
                corr_2dlist = str22dlist(kp_coor)
                for kp_idx, corr in enumerate(corr_2dlist):
                    kp_data = [int(row_data[3]["ScalarValue"])-1, kp_label[row_data[5]["ScalarValue"]],
                               int(row_data[6]["ScalarValue"]), kp_idx,
                               row_data[4]["ScalarValue"], row_data[2]["ScalarValue"],
                               corr[0], corr[1],
                               1.0]
                    kp_data_list.append(kp_data)
            else:
                print('data is not kp or bb')
        else:
            break
#     print(kp_data_list)
    bbox_df = pd.DataFrame.from_records(bbox_data_list, columns=['frameId', 'label', 'bbox_idx', 'top_left_x', 
                                                             'top_left_y', 'width', 'height', 'OCR_ID', 
                                                             'time', 'conf'])
    print(bbox_df.info(verbose=True))
    bbox_df = bbox_df.sort_values(by=['frameId', 'label', 'bbox_idx'])
    bbox_df.to_csv('csv_data/'+video_name+'_bbox.csv',index=False)
    kp_df = pd.DataFrame.from_records(kp_data_list, columns=['frameId', 'label', 'bbox_idx', 'kp_idx', 'OCR_ID', 
                                                             'time', 'x', 'y', 'conf'])
    print(kp_df.info(verbose=True))
    kp_df = kp_df.sort_values(by=['frameId', 'label', 'bbox_idx'])
    kp_df.to_csv('csv_data/'+video_name+'_kp.csv',index=False)
# frameId,label,bbox_idx,kp_idx,x,y,OCR_ID,time, conf
# frameId,label,bbox_idx,top_left_x,top_left_y,width,height,OCR_ID,time, conf
# "ColumnInfo": [
#     { "Name": "measure_value::varchar", "Type": { "ScalarType": "VARCHAR" } },
#     { "Name": "measure_name", "Type": { "ScalarType": "VARCHAR" } },
#     { "Name": "time", "Type": { "ScalarType": "TIMESTAMP" } },
#     { "Name": "Frame", "Type": { "ScalarType": "VARCHAR" } },
#     { "Name": "OCR_ID", "Type": { "ScalarType": "VARCHAR" } },
#     { "Name": "Class", "Type": { "ScalarType": "VARCHAR" } },
#     { "Name": "ID", "Type": { "ScalarType": "VARCHAR" } },
#     { "Name": "File", "Type": { "ScalarType": "VARCHAR" } }
#   ],

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 789 entries, 0 to 788
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   frameId     789 non-null    int64  
 1   label       789 non-null    object 
 2   bbox_idx    789 non-null    int64  
 3   top_left_x  789 non-null    int64  
 4   top_left_y  789 non-null    int64  
 5   width       789 non-null    int64  
 6   height      789 non-null    int64  
 7   OCR_ID      789 non-null    object 
 8   time        789 non-null    object 
 9   conf        789 non-null    float64
dtypes: float64(1), int64(6), object(3)
memory usage: 61.8+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10294 entries, 0 to 10293
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   frameId   10294 non-null  int64  
 1   label     10294 non-null  object 
 2   bbox_idx  10294 non-null  int64  
 3   kp_idx    10294 non-null  i